## 一、介绍
使用2001-2021年的mda数据(1.45G)，训练出的中国A股市场词向量模型，词汇量789539， 模型文件650M。可广泛用于经济管理等领域概念(情感)词典的构建或扩展。 

训练环境windows256G内存， 2.0.0版本的cntext库。在该环境下，我也尝试使用14G的年报数据，训练了两天，跑不出结果，256G的内存基本用光了。所以cntext训练模型，适合的数据规模是1G左右。需要注意的是，目前网上公开的cntext版本是1.8.4， 有三个原因暂时不开源2.0.0:

- 目前我的自由时间不太充裕，公开2.0.0版本，我需要重新录制付费视频课程。
- 1.8.4及之前版本也能训练词向量模型，只是代码简洁性方面略微不如2.0.0
- 可能也是最重要的是，2.0.0综合了训练语言模型&多语言模型对齐， 有较大的应用价值。
  - 可以测量同主体不同时间段大样本内蕴含的文化变迁
  - 或同时间段，不同主体的大样本文本蕴含的差异性
  

<br>

扯远了，回归正题，模型文件

- mda01-21.200.6.bin  
- mda01-21.200.6.bin.vectors.npy


参数解读

- mda01-21 使用2001-2021年度的mda数据训练
- 200 嵌入的维度数，即每个词的向量长度是200
- 6 词语上下文的窗口是6


<br>
<br>

## 二、导入模型
需要用到两个自定义函数load_w2v、expand_dictionary，源代码太长，为了提高阅读体验， 放在文末。大家记得用这两个函数前一定要先导入

In [47]:
#先导入load_w2v、expand_dictionary函数源代码


#读取模型文件
wv = load_w2v(w2v_path='model/mda01-21.200.6.bin')
wv

Loading word2vec model...


### 三、wv的使用
- 查看词汇量
- 查询某词向量
- 查看多个词的均值向量

更多内容，建议查看下gensim库的文档

In [48]:
#词汇量
len(wv.index_to_key)

789539

In [49]:
#查询某词的词向量
wv.get_vector('创新')

array([ 4.34389877e+00, -4.93447453e-01,  2.17293240e-02,  1.90846980e+00,
        8.75901580e-01, -7.95542181e-01, -1.12950909e+00,  7.44228721e-01,
        7.38122821e-01,  6.42377853e-01,  3.99175316e-01,  2.17924786e+00,
        9.30410504e-01, -3.23538423e+00, -2.91860670e-01,  1.04046893e+00,
       -1.73857129e+00, -1.12141383e+00,  3.51870751e+00, -8.69141936e-01,
        4.95228887e-01,  4.80194688e-01, -3.35257435e+00,  7.16054797e-01,
        2.29016230e-01,  2.40962386e+00, -7.40825295e-01,  2.18998361e+00,
       -3.37587762e+00, -1.30376315e+00,  5.08445930e+00, -1.68504322e+00,
       -1.60081315e+00, -8.33779454e-01, -7.58818448e-01, -1.78838921e+00,
        2.44672084e+00,  2.27579999e+00, -2.52457595e+00,  1.36214256e-01,
       -3.09675723e-01, -6.98232710e-01,  1.73018420e+00, -8.05342972e-01,
       -1.70148358e-01, -2.43612671e+00, -1.23085886e-01,  2.83124876e+00,
        3.89446110e-01, -3.16048344e-03, -2.09607935e+00, -1.49788404e+00,
        8.58029604e-01, -

In [50]:
#查询多个词的词向量
wv.get_mean_vector(['创新', '研发'])

array([ 0.17623448, -0.02220692, -0.01040847,  0.03616136,  0.04931263,
       -0.06220303, -0.02846557, -0.00156435,  0.04524047,  0.03185674,
        0.01104859,  0.06962118, -0.01969986, -0.10831943, -0.0524368 ,
        0.00623383, -0.04149605, -0.004912  ,  0.13154642, -0.04317038,
       -0.00407438,  0.00923527, -0.13339072,  0.01446994, -0.00153984,
        0.12378754, -0.06064663,  0.09322313, -0.07711462, -0.05880795,
        0.13697049,  0.0133168 ,  0.02769322,  0.02677607,  0.02549294,
       -0.04504526,  0.06267191,  0.02421109, -0.13401456,  0.01423616,
        0.01860182,  0.00344108,  0.04811918,  0.02748652,  0.0190251 ,
       -0.03800797,  0.01517046,  0.06439836,  0.01320594,  0.04748138,
       -0.08914943, -0.00642068,  0.01786153, -0.02472607, -0.04597819,
        0.05832303,  0.11275461, -0.0387079 ,  0.06912261,  0.05287468,
       -0.04447906,  0.10994074, -0.04371417,  0.01227543,  0.07498093,
       -0.11285575, -0.03113984, -0.01122221, -0.03913497, -0.12

有了每个词或者概念的向量，可以结合cntext旧版本单语言模型内的态度偏见的度量。

<br>

## 四、扩展词典
做词典法的文本分析，最重要的是有自己的领域词典。之前受限于技术难度，文科生的我也一直在用形容词的通用情感词典。现在依托word2vec技术， 可以加速人工构建的准确率和效率。


下面是在 mda01-21.200.6.bin 上做的词典扩展测试，函数expand_dictionary会根据种子词选取最准确的topn个词。

In [45]:
#短视主义词  实验
expand_dictionary(wv=wv, 
                  seedwords=['抓紧', '立刻', '月底', '年底', '年终', '争取', '力争'],
                  topn=30)

['抓紧',
 '立刻',
 '月底',
 '年底',
 '年终',
 '争取',
 '力争',
 '争取',
 '力争',
 '年内',
 '月底',
 '年底',
 '尽早',
 '3月底',
 '尽快',
 '抓紧',
 '6月份',
 '4月份',
 '月份',
 '工作力争',
 '努力争取',
 '工作争取',
 '10月底',
 '年内实现',
 '年底完成',
 '中旬',
 '7月份',
 '9月底',
 '有望',
 '月底前',
 '早日',
 '全力',
 '继续',
 '月初',
 '努力',
 '确保',
 '8月份']

In [26]:
expand_dictionary(wv=wv, 
                  seedwords=['团结', '拼搏',  '克服',  '勇攀高峰',  '友善',  '进取'],
                  topn=30)

['团结',
 '拼搏',
 '克服',
 '勇攀高峰',
 '友善',
 '进取',
 '拼搏',
 '艰苦奋斗',
 '坚定信念',
 '团结拼搏',
 '上下同心',
 '团结',
 '顽强拼搏',
 '勇于担当',
 '团结一致',
 '团结奋进',
 '精诚团结',
 '齐心协力',
 '开拓进取',
 '奋进',
 '团结一心',
 '实干',
 '同心协力',
 '团结协作',
 '锐意进取',
 '积极进取',
 '奋力拼搏',
 '拼搏精神',
 '努力拼搏',
 '进取',
 '奋发有为',
 '扎实工作',
 '同心同德',
 '拼搏进取',
 '脚踏实地',
 '励精图治']

In [27]:
expand_dictionary(wv=wv, 
                  seedwords=['创新', '科技',  '研发',  '技术',  '标准'],
                  topn=30)

['创新',
 '科技',
 '研发',
 '技术',
 '标准',
 '创新',
 '技术创新',
 '技术研发',
 '科技创新',
 '先进技术',
 '自主创新',
 '前沿技术',
 '关键技术',
 '科研',
 '新技术',
 '创新性',
 '研发创新',
 '产品研发',
 '基础研究',
 '产品开发',
 '集成创新',
 '核心技术',
 '自主研发',
 '技术应用',
 '技术集成',
 '前沿科技',
 '技术标准',
 '工艺技术',
 '科技成果',
 '技术开发',
 '尖端技术',
 '工程技术',
 '技术相结合',
 '科学技术',
 '工艺']

In [28]:
expand_dictionary(wv=wv, 
                  seedwords=['竞争', '竞争力'],
                  topn=30)

['竞争',
 '竞争力',
 '竞争能力',
 '竞争优势',
 '市场竞争',
 '竞',
 '市场竞争力',
 '竞争实力',
 '参与市场竞争',
 '国际竞争',
 '市场竞争能力',
 '核心竞争力',
 '激烈竞争',
 '市场竞争优势',
 '竞争态势',
 '参与竞争',
 '竞争力重要',
 '竞争对手',
 '创新能力',
 '综合竞争力',
 '价格竞争',
 '之间竞争',
 '核心竞争能力',
 '未来市场竞争',
 '国际竞争力',
 '影响力竞争力',
 '国际化竞争',
 '行业竞争',
 '综合竞争能力',
 '竞争日趋激烈',
 '产品竞争力',
 '竞争力影响力']

In [44]:
expand_dictionary(wv=wv, 
                  seedwords=['疫情', '扩散', '防控', '反复', '冲击'],
                  topn=30)

['疫情',
 '扩散',
 '防控',
 '反复',
 '冲击',
 '蔓延',
 '疫情冲击',
 '疫情爆发',
 '新冠疫情',
 '新冠肺炎',
 '疫情蔓延',
 '疫情暴发',
 '肆虐',
 '本次疫情',
 '冲击疫情',
 '新冠病毒',
 '疫情扩散',
 '全球蔓延',
 '疫情影响',
 '病毒疫情',
 '肺炎疫情',
 '击',
 '持续蔓延',
 '疫情持续',
 '各地疫情',
 '疫情突然',
 '疫情全球',
 '疫情传播',
 '疫情反复',
 '散发',
 '变异毒株',
 '疫情导致',
 '疫情肆虐',
 '全球疫情',
 '全球新冠']

In [31]:
expand_dictionary(wv=wv, 
                  seedwords=['旧', '老', '后', '落后'],
                  topn=30)

['旧',
 '老',
 '后',
 '落后',
 '旧',
 '老',
 '陈旧',
 '老旧',
 '淘汰',
 '高能耗',
 '低效率',
 '设备陈旧',
 '能耗高',
 '老旧设备',
 '落后工艺',
 '进行改造',
 '工艺落后',
 '技术落后',
 '翻新',
 '更新改造',
 '改造',
 '更新',
 '替换',
 '改造更新',
 '旧设备',
 '污染重',
 '淘汰一批',
 '拆除',
 '污染严重',
 '简陋',
 '产能落后',
 '相对落后',
 '产能淘汰',
 '效率低下']

In [46]:
from gensim.models import KeyedVectors
from pathlib import Path


def load_w2v(w2v_path):
    """
    Load word2vec model

    Args:
        w2v_path (str): path of word2vec model

    Returns:
        model: word2vec model
    """
    print('Loading word2vec model...')
    model = KeyedVectors.load(w2v_path)
    return model


def expand_dictionary(wv, seedwords, topn=100):
    """
    According to the seed word file, select the top n words with the most similar semantics and save them in the directory save_dir.
    
    Args:
        wv (Word2VecKeyedVectors): the word embedding model
        seedwords (list): 种子词
        topn (int, optional): Set the number of most similar words to retrieve to topn. Defaults to 100.
        save_dir (str, optional): the directory to save the candidate words. Defaults to 'Word2Vec'.
    
    Returns:
    """
    simidx_scores = []

    similars_candidate_idxs = [] #the candidate words of seedwords
    dictionary = wv.key_to_index
    seedidxs = [] #transform word to index
    for seed in seedwords:
        if seed in dictionary:
            seedidx = dictionary[seed]
            seedidxs.append(seedidx)
    for seedidx in seedidxs:
        # sims_words such as [('by', 0.99984), ('or', 0.99982), ('an', 0.99981), ('up', 0.99980)]
        sims_words = wv.similar_by_word(seedidx, topn=topn)
        #Convert words to index and store them
        similars_candidate_idxs.extend([dictionary[sim[0]] for sim in sims_words])
    similars_candidate_idxs = set(similars_candidate_idxs)
    
    for idx in similars_candidate_idxs:
        score = wv.n_similarity([idx], seedidxs)
        simidx_scores.append((idx, score))
    simidxs = [w[0] for w in sorted(simidx_scores, key=lambda k:k[1], reverse=True)]

    simwords = [str(wv.index_to_key[idx]) for idx in simidxs][:topn]

    resultwords = []
    resultwords.extend(seedwords)
    resultwords.extend(simwords)
    
    return resultwords